In [21]:
%matplotlib inline
%pylab inline
import os
from tqdm import tqdm

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import pylab as pl

from Bio import SeqIO

Populating the interactive namespace from numpy and matplotlib


Loading files:

In [2]:
record_dict = SeqIO.index("data/uniprot_sprot.fasta", "fasta")
# record_dict = SeqIO.to_dict(SeqIO.parse("data/uniprot_sprot.fasta", "fasta"))

In [4]:
list(record_dict.keys())[:9]

['sp|A6MVV7|PETL_RHDSA',
 'sp|Q2A4C9|COAX1_FRATH',
 'sp|Q89AI5|TSAB_BUCBP',
 'sp|Q9CEF5|PKNB_LACLA',
 'sp|Q1CDT5|FIS_YERPN',
 'sp|B0TWS7|ATPB_FRAP2',
 'sp|Q3AV54|ACCA_SYNS9',
 'sp|P24966|CYB_PECTA',
 'sp|P39061|COIA1_MOUSE']

In [18]:
i = 2
len(record_dict['sp|A6MVV7|PETL_RHDSA'].seq[i:i+9])

9

In [24]:
filename = 'output/human_peptides.txt'
target = open(filename, 'w')

In [1]:
# for key in tqdm(record_dict):
#     lines = []
#     for i in range(len(record_dict[key].seq)-9):
#         lines.append(str(record_dict[key].seq[i:i+9]))
#     for i in range(len(lines)):
#         target.write(lines[i])
#         target.write('\n')